# C3D(으)로 features 뽑아내기
https://github.com/dolongbien/HumanBehaviorBKU/tree/master/C3D

In [ ]:
!nvidia-smi

Mon Sep 21 07:12:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from tensorflow.python.client import device_lib 
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4801479299141087943, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 792746219511988133
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2434694289217096327
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11146722048
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3858014125402223734
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [ ]:
# check Utubtu version
!lsb_release -a
# by default, CUDA is enabled under GPU mode of Google Colab
!ls /usr/local

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic
bin	   cuda-10.1  _gcs_config_ops.so  LICENSE.txt  setup.cfg  xgboost
cuda	   etc	      include		  man	       share
cuda-10.0  games      lib		  sbin	       src


In [ ]:
# optional
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Note_1
As of the CUDA 8 release, gcc 5.3 is fully supported on Ubuntu 16.06 and Fedora 23.   
As of the CUDA 9 release, gcc 6 is fully supported on Ubuntu 16.04, Ubuntu 17.04 and Fedora 25.   
The CUDA 9.2 release adds support for gcc 7

In [ ]:
# make g++, nvcc and boost work together (compatible). 
# Better install gcc before anything else
# update-alternatives --install [link] [name] [path] [priority]

!sudo apt update
!sudo apt install g++-5
!sudo update-alternatives --remove-all gcc 
!sudo update-alternatives --remove-all g++

!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 20
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-5 20

!sudo update-alternatives --install /usr/bin/cc cc /usr/bin/gcc 30
!sudo update-alternatives --set cc /usr/bin/gcc

!sudo update-alternatives --install /usr/bin/c++ c++ /usr/bin/g++ 30
!sudo update-alternatives --set c++ /usr/bin/g++

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,876

In [ ]:
!sudo apt-get install -f

# install dependencies

!sudo apt-get install -y libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
!sudo apt-get install libatlas-base-dev
!sudo apt-get install -y --no-install-recommends libboost-all-dev
!sudo apt-get install -y libgflags-dev libgoogle-glog-dev liblmdb-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following additional packages will be installed:
  libleveldb1v5 libprotobuf-lite10
Suggested packages:
  leveldb-doc
The following NEW packages will be installed:
  libhdf5-serial-dev libleveldb-dev libleveldb1v5 libprotobuf-dev
  libprotobuf-lite10 libsnappy-dev
0 upgraded, 6 newly installed, 0 to remove and 39 not upgraded.
Need to get 1,400 kB of archives.
After this operation, 9,964 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libhdf5-serial-dev all 1.10.0-patch1+docs-4 [2,898 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libleveldb1v5 amd64 1.20-2 [136 kB]
Get:3 http://archive.ubuntu.com

# Note_2
CUDA, OPENCV
- BLAS (Basic Linear Algebra Subprograms): operations like matrix
multiplication, matrix addition, both implementation for CPU(cBLAS) and
GPU(cuBLAS). provided by MKL(INTEL), ATLAS, openBLAS, etc.
- Boost: a c++ library. > Use some of its math functions and shared_pointer.
- glog,gflags provide logging & command line utilities. > Essential for
debugging.
- leveldb, lmdb: database io for your program. > Need to know this for
preparing your own data.
- protobuf: an efficient and flexible way to define data structure. > Need to
know this for defining new layer

In [ ]:
%cd /content
!git clone https://github.com/dolongbien/C3D.git

/content
Cloning into 'C3D'...
remote: Enumerating objects: 1737, done.
remote: Total 1737 (delta 0), reused 0 (delta 0), pack-reused 1737
Receiving objects: 100% (1737/1737), 28.62 MiB | 16.94 MiB/s, done.
Resolving deltas: 100% (597/597), done.


In [ ]:
# compile

%cd /content/C3D/C3D-v1.0/
!make clean
!make all -j8

/content/C3D/C3D-v1.0
protoc --proto_path=src --cpp_out=build/src src/caffe/proto/caffe.proto
protoc --proto_path=src --cpp_out=build/src src/caffe/proto/caffe_pretty_print.proto
/usr/local/cuda/bin/nvcc -ccbin=/usr/bin/g++ -Xcompiler -fPIC -DNDEBUG -O2 -I/usr/local/include/python2.7 -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -I/usr/local/include -I/usr/include/hdf5/serial/ -Ibuild/src -I./src -I./include -I/usr/local/cuda/include -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode=arch=compute_50,code=sm_50 -gencode=arch=compute_37,code=sm_37  -c src/caffe/util/math_functions.cu -o build/src/caffe/util/math_functions.cuo
/usr/local/cuda/bin/nvcc -ccbin=/usr/bin/g++ -Xcompiler -fPIC -DNDEBUG -O2 -I/usr/local/include/python2.7 -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -I/usr/local/include -I/usr/include/hdf5/serial/ -Ibuild/src -I./src -I./include -I/usr/local/cuda/include -gencode arch=compute_30,code=sm_30 -gencode arch=c

In [ ]:
!make test

/usr/bin/g++ src/caffe/test/test_power_layer.cpp -pthread -fPIC -DNDEBUG -O2 -I/usr/local/include/python2.7 -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -I/usr/local/include -I/usr/include/hdf5/serial/ -Ibuild/src -I./src -I./include -I/usr/local/cuda/include -c -o build/src/caffe/test/test_power_layer.o

/usr/bin/g++ src/caffe/test/test_util_blas.cpp -pthread -fPIC -DNDEBUG -O2 -I/usr/local/include/python2.7 -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -I/usr/local/include -I/usr/include/hdf5/serial/ -Ibuild/src -I./src -I./include -I/usr/local/cuda/include -c -o build/src/caffe/test/test_util_blas.o

/usr/bin/g++ src/caffe/test/test_filler.cpp -pthread -fPIC -DNDEBUG -O2 -I/usr/local/include/python2.7 -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -I/usr/local/include -I/usr/include/hdf5/serial/ -Ibuild/src -I./src -I./include -I/usr/local/cuda/include -c -o build/src/caffe/test/test_filler.o

/usr/bin/g++ src/caffe/test/test_euclidean_loss_

In [ ]:
# 이건 오래걸리기도 하고 옵션으로 하는거라서 시간상 건너뛰자
# optional
# !make runtest

In [ ]:
%cd /content/C3D/C3D-v1.0/examples/c3d_feature_extraction/
!wget -O conv3d_deepnetA_sport1m_iter_1900000 https://www.dropbox.com/s/vr8ckp0pxgbldhs/conv3d_deepnetA_sport1m_iter_1900000?dl=0

/content/C3D/C3D-v1.0/examples/c3d_feature_extraction
--2020-09-21 08:00:15--  https://www.dropbox.com/s/vr8ckp0pxgbldhs/conv3d_deepnetA_sport1m_iter_1900000?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.1, 2620:100:6023:1::a27d:4301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/vr8ckp0pxgbldhs/conv3d_deepnetA_sport1m_iter_1900000 [following]
--2020-09-21 08:00:16--  https://www.dropbox.com/s/raw/vr8ckp0pxgbldhs/conv3d_deepnetA_sport1m_iter_1900000
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9effeec3bbe500412acc3467b4.dl.dropboxusercontent.com/cd/0/inline/A_w40rnpC0jQ57xr-R1U7N9JbR5NVwk6C8O0pfvbX8dfDeZE9XGLpqkpx7jY_hqU3LEjiO75GgyShUW5QbXXx5uxdT1qKjVG66kNnZq5-lgl7w/file# [following]
--2020-09-21 08:00:16--  https://uc9effeec3bbe500412acc3467b4.dl.dropboxusercontent.com/cd/0/inline/A

# Extract feature, check output folder in c3d_sport1m_feature_extraction_video.sh

## fc6 features를 뽑는 곳

In [ ]:
# extract feature
%cd /content/C3D/C3D-v1.0/examples/c3d_feature_extraction
!sudo bash c3d_sport1m_feature_extraction_video.sh

/content/C3D/C3D-v1.0/examples/c3d_feature_extraction
E0921 08:00:50.039165  6950 extract_image_features.cpp:64] Using GPU #0
I0921 08:00:50.040972  6950 net.cpp:76] Creating Layer data
I0921 08:00:50.041011  6950 net.cpp:112] data -> data
I0921 08:00:50.041079  6950 net.cpp:112] data -> label
I0921 08:00:50.041245  6950 video_data_layer.cpp:283] Opening file prototxt/input_list_video.txt
I0921 08:00:50.041355  6950 video_data_layer.cpp:317] A total of 16 video chunks.
I0921 08:00:50.172540  6950 video_data_layer.cpp:365] output data size: 50,3,16,112,112
I0921 08:00:50.172572  6950 video_data_layer.cpp:387] Loading mean file from sport1m_train16_128_mean.binaryproto
I0921 08:00:50.185624  6950 net.cpp:127] Top shape: 50 3 16 112 112 (30105600)
I0921 08:00:50.185647  6950 net.cpp:127] Top shape: 50 1 1 1 1 (50)
I0921 08:00:50.185655  6950 net.cpp:159] data does not need backward computation.
I0921 08:00:50.185669  6950 net.cpp:76] Creating Layer conv1a
I0921 08:00:50.185675  6950 net.c

# generate_inout_file.py

## 다른 동영상으로 fc6을 뽑고 싶을때 이 과정을 거쳐야함

In [ ]:
# %cd /content/gdrive/My\ Drive/C3D/C3D-v1.0/examples/c3d_feature_extraction/input/Traning_Normal_Part1_Video
%cd /content/C3D/C3D-v1.0/examples/c3d_feature_extraction/input/Traning_Normal_Part1_Video
import os, os.path
import cv2
import glob
import math

f_input = open('input_list_video.txt', 'w')
f_output = open('output_list_video_prefix.txt','w')
sh_out = open('c3d_sport1m_feature_extraction_video.sh', 'w')

interval_v = 16

cmd1 = 'time GLOG_logtosterr=1 ../../build/tools/extract_image_features.bin prototxt/c3d_sport1m_feature_extractor_video.prototxt conv3d_deepnetA_sport1m_iter_1900000'

cmd2 = 'prototxt/output_list_video_prefix.txt fc6-1'

gpu_id = 0
batch_size = 50
batch_num = 1
max_batch_num = 1

# read file path name in alphabet order
file_list_v = sorted(glob.glob("*.mp4"))

# each video, generate input/output file
for index in range(len(file_list_v)):
	
	# file path of video
	loc = file_list_v[index]
	cap=cv2.VideoCapture(loc)
	
	# generate .sh file
	sh_out.write('rm -rf output/Traning_Normal_Part1_FC/' + loc[:-4])
	sh_out.write('\n')
	sh_out.write('mkdir -p output/Traning_Normal_Part1_FC/' + loc[:-4])
	sh_out.write('\n')
	
	# number of frames
	total_frames = int(cap.get(7))
	print(index+1, total_frames, sep='\tframes=')
	
	# calculate batch number
	curr_batch_num = math.ceil(total_frames/batch_size)
	max_batch_num = batch_num
	if curr_batch_num >= max_batch_num:
		batch_num = curr_batch_num
	else:
		batch_num = max_batch_num
	

	counter = 0   

	# generate file
	while (counter <= total_frames-interval_v):
		f_input.write('input/Traning_Normal_Part1_Video/' + loc + ' ' + str(counter) + ' ' + str(0))
		f_input.write('\n')
		f_output.write('output/Traning_Normal_Part1_FC/' + loc[:-4] + '/' + str(counter).zfill(6) )
		f_output.write('\n')
		counter = counter + interval_v

# write command to bash file
parameter = ' ' + str(gpu_id) + ' ' + str(batch_size) + ' ' + str(batch_num) + ' ' 
sh_out.write(cmd1 + parameter + cmd2)

print('-------- PARAMETER --------')
print('GPU ID is', gpu_id)
print('Batch size is', batch_size)
print('Number of batch is', batch_num)

f_input.close()
f_output.close()
sh_out.close()

/content/C3D/C3D-v1.0/examples/c3d_feature_extraction/input/Traning_Normal_Part1_Video
1	frames=1415
2	frames=1034
3	frames=1457
4	frames=1822
5	frames=1676
6	frames=969
7	frames=585
8	frames=1220
9	frames=2344
10	frames=849
-------- PARAMETER --------
GPU ID is 0
Batch size is 50
Number of batch is 47


# read_featureFC_file.py 응용

## 1. 실패

In [ ]:
# clc() ##command창에 표시된 코드들 사라지게해
#-*- encoding: utf8 -*-
import os
import numpy as np
import struct
# clear(mstring('all'))
# close(mstring('all'))

# This code save already computed C3D features into 32 (video features) segments.
# We assume that C3D features for a video are already computed. We use
# default settings for computing C3D features, i.e., we compute C3D features for
# every 16 frames and obtain the features from fc6.

import struct

def read_bin(input_file):
    # input_file = open(r'/content/gdrive/My Drive/folder/v_ApplyEyeMakeup_g01_c01/000000.fc6-1','rb')
    input_file = open(input_file, 'rb')
    
    try:
        sizes = [struct.unpack('i',input_file.read(4))[0] for i in range(5)]
        m = np.prod(sizes)
        data = [struct.unpack('f',input_file.read(4))[0] for i in range(m)]
    finally:
        input_file
    feature_vector = np.array(data)

    return feature_vector, feature_vector.shape

C3D_Path = r'/content/gdrive/My Drive/folder'
C3D_Path_Seg = r'/content/C3D/C3D-v1.0/examples/c3d_feature_extraction/output/make_segment'

if not os.path.isdir(C3D_Path_Seg):
    os.mkdir(C3D_Path_Seg)

print('DONE')

All_Folder = os.listdir(C3D_Path)
# All_Folder = All_Folder[3:end]
subcript = '_C.txt'

for ifolder in All_Folder:
    #% START 1 LOOP WITH 1 FC FOLDER, ex: Abuse028 has N=1392 frames

    Folder_Path = str(C3D_Path) + "/" + str(ifolder)
    #Folder_Path is path of a folder which contains C3D features (for every 16 frames) for a paricular video.
    # N=1392 frames --> it has [1392/16] = 88 fc6-1 files

    AllFiles = os.listdir(Folder_Path) ##"/.fc6-1 확장자 파일 싹다 리스트로 반환"
    # fc6-1 files in feature directory, each file = a clip in video
    # one clip = 16 frames

    if len(AllFiles) == 0:
        print("no fc6-1 file in path")
        continue
        
    feature_vector = np.zeros((len(AllFiles), 4096))
    # each fc6-1 = 1 clips 16 frames = 4096-d ==> Total is [N/16]=88 clips like that
    #% Iterate each fc6-1 file (16 frames each)
    for ifile in range(0,len(AllFiles)):
        FilePath =Folder_Path + '/' + AllFiles[ifile]

        data,_ = read_bin(FilePath)
        _,s = read_bin(FilePath)
        feature_vector[ifile]=data #% 1 column 4096-d in 88x4096 is assign by 1 clip feature (4096)

        # clear(mstring('data')) # clear라는 변수를 매트랩 내에서 삭제

    #% At this point, Feature vector is filled with all actual data from
    # all 16-frame clips in video, each clip is 4096-d, therefore 88x4096
    # is now filled with actual data
    # if sum(sum(feature_vector, [])) == 0: ## 고쳐야됨 : 각 열의 합인 한 행짜리 행렬로
    #     print('error1')

        # Write C3D features in text file to load in
        # Training_AnomalyDetector_public ( You can directly use .mat format if you want).
        txt_file_name = C3D_Path_Seg + '/' + ifolder +subcript
        # feature txt name i.e Abuse028_x264_C.txt

    # if exist(txt_file_name, 'file'):
    #     continue

    fid1 = open(txt_file_name, 'w')
    ## sum(x,1) = sum vertically (column)
    ## sum(x,2) = sum horizontally (row)
    # if not isempty(find(sum(Feature_vect, 2) == 0)): # sum row --> 88x4096 results in 88 rows
    #     # k = find(X,n)은 X의 0이 아닌 요소에 대응하는 처음 n개의 인덱스를 반환합니다
    #     print('error2')


    # if not isempty(find(isnan(Feature_vect(mslice[:])))):
    #     print('error3')

    # if not isempty(find(Feature_vect(mslice[:]) == Inf)):
    #     print('error4')

        #% 32 Segments

    Segments_Features = np.zeros((32, 4096))    #32 row, 4096 column
    thirty2_shots = np.linspace(1, len(AllFiles), 33).round(0)
    # thirty2shots = divide 88 frames to 33 segment, start from 1 to 88
    # SO: thirty2shots = [1 , 4, 6, 10, ..... 83, 85, 88], total elements
    # is 33, vector 1x33
    count = -1
    #% WRITE 88x4096 TO 32x4096
    for ishots in range(0,len(thirty2_shots) - 1):                                   # ishorts starts from 1 to 32
        ss = int(thirty2_shots[ishots] )                                   # start clip index in 88x4096
        ee = int(thirty2_shots[ishots + 1] - 1)                             # end clip index in 88x4096
        
        # print(ss,ee,'llllll')
        if ishots == len(thirty2_shots):
            ee = int(thirty2_shots[ishots + 1])
            #% THIS IS A FEATURE FOR 1 SEGMENT
            #ALL BELOW CASE, temp_vect is always 4096-d based on value of start ss and end ee index
        if ss == ee:
            temp_vect = feature_vector[ss] ##ss번째 행 벡터 추출     # ss==ee --> get 1 vector 4096-d from 88x4096

        elif ee < ss:
            temp_vect = feature_vector[ss]  # ee < ss --> get 1 vector 4096-d from 88x4096

        else:
            temp_vect = feature_vector[ss:ee].mean(axis=0) ##각 열의 평균값을 가진 1*4096행 추출
            # for i in range(ss,ee):
                # feature_vector
            # ss < ee --> get all clip vectors from ss to ee (ex: 3 vectors) from 88x4096
            # origin feature, than take mean value of all (i.e 3 vectors) that vectors to
            # get a new one has 4096-d (shape of result is shape of row when get mean a
            # matrix)
            #mean a vector = mean of each column = sum column/total row -->
            #shape = number of row (=4096) after this mean operation
        # print(temp_vect.shape)
        #% AFTER HAS 1 SEGMENT FEATURE,  CALCULATE NORM-2 (L2)
        temp_vect = temp_vect / np.linalg.norm(temp_vect)
        # temp_vect = temp_vect / np.norm(temp_vect)   #% divide by norm-2 (L2) of vector (Euclidean norm)=cumsum(sqrt(x[i]^2))                                     # divide by norm-2 (L2) of vector (Euclidean norm)=cumsum(sqrt(x[i]^2))

        # if np.linalg.norm(temp_vect) == 0:
        #     print('error5')

        count = count + 1 # next segment (max=32)
        Segments_Features[count]= temp_vect # push each segment feature to final 32 video segments feature

#verify

    # if not isempty(find(sum(Segments_Features, 2) == 0)):
    #     print('error6')

    # if not isempty(find(isnan(Segments_Features(mslice[:])))):
    #     print('error7')


    # if not isempty(find(Segments_Features(mslice[:]) == Inf)):
    #     print('error8')


        # save 32 segment features in text file ( You can directly save and load .mat file in python as well).
    print(Segments_Features)
    print(Segments_Features.shape)

    for i in range(0,Segments_Features.shape[0]):
        feat_text = str(Segments_Features[i].tolist())
        fid1.write(feat_text)
        fid1.write('\n')

    fid1.close()

DONE


IndexError: ignored

## 2. 성공

In [ ]:
import os
import numpy as np
import struct

    ## <read_featureFC_file.py>(from 도롱빈)================================
def read_bin(input_file):
# input_file = open(r'.\normal_video_001\000032.fc6-1','rb')
    input_file = open(input_file,'rb')
    try:
        sizes = [struct.unpack('i',input_file.read(4))[0] for i in range(5)]
        m = np.prod(sizes)
        data = [struct.unpack('f',input_file.read(4))[0] for i in range(m)]
    finally:
        input_file
    feature_vector = np.array(data)

    return feature_vector, feature_vector.shape

    ## ======================================================================

C3D_Path = r'/content/gdrive/My Drive/folder'
C3D_Path_Seg = r'/content/C3D/C3D-v1.0/examples/c3d_feature_extraction/output/make_segment'


All_Folder = os.listdir(C3D_Path)
subcript = '_C.txt'

for ifolder in All_Folder:
    print(ifolder)
    # START 1 LOOP WITH 1 FC FOLDER(영상 1개)

    Folder_Path = str(C3D_Path) + "/" + str(ifolder)
    AllFiles = os.listdir(Folder_Path) # /.fc6-1 확장자 파일 싹다 리스트로 반환
                                       # fc6-1 files in feature directory, each file = a clip in video
                                       # one clip = 16 frames

    if len(AllFiles) == 0:
        print("no fc6-1 file in path")
        continue
    
    # .fc6-1 읽고 feature_vector 생성
    feature_vector = np.zeros((len(AllFiles), 4096))
    # each fc6-1 = 1 clips 16 frames = 4096-d ==> Total is [N/16]=88 clips like that
    # Iterate each fc6-1 file (16 frames each)
    for ifile in range(0,len(AllFiles)):
        FilePath =Folder_Path + '/' + AllFiles[ifile]

        data, s = read_bin(FilePath)
        feature_vector[ifile]=data # 1 row 4096-d in 88x4096 is assign by 1 clip feature (4096)
        # 이제 fc6-1을 모두 읽어서 feature_vector(shape = (fc6-1 files 개수) x 4096)에 넣음


#----------------------------------------------------------------------------------
# 여기서부터 feature vector를 32seg로 나눔
# ((fc6-1 files 개수), 4096) -> (32, 4096)으로 만들어서 새로운 파일로 저장
# 여기서 만든 파일을 Training_AnomalyDetector_public.py에 넣어서 training 시킬 수 있음

    txt_file_name = C3D_Path_Seg + '/' + ifolder +subcript
    # feature txt name i.e abnormal_video_001_C.txt

    # fid1 = open(txt_file_name, 'w')

    sum_row = np.sum(feature_vector, axis=1) # 각 16 frame clip에 해당하는 row를 sum
    if np.where(sum_row==0)[0].size != 0:
        print('Error2')
    if np.isnan(feature_vector).sum() != 0:
        print('Error3')
    if np.inf in feature_vector:
        print('Error4')
 

    # 32 Segments
    Segments_Features = np.zeros((32, 4096))    # (32, 4096)
    thirty2_shots = np.linspace(0, len(AllFiles)-1, 33).round(0).astype(int) # AllFiles = .fc-1 확장자 파일들
    # SO: thirty2shots = [1 , 4, 6, 10, ..... 83, 85, 88], total elements is 33, vector 1x33
    
    count = -1
    # WRITE (88, 4096) TO (32, 4096)
    for ishots in range(0,len(thirty2_shots) - 1):  # ishorts starts from 0 to 31 (총 32개)
        ss = int(thirty2_shots[ishots])            # start clip index 
        ee = int(thirty2_shots[ishots + 1] - 1)     # end clip index 
        

        if ss == ee: # thirty2_shots에서 값의 차이가 1일 때
            temp_vect = feature_vector[ss] # ss==ee --> get 1 vector 4096-d from 88x4096

        elif ee < ss: # 영상 frame이 적어서 thirty2_shots에 같은 숫자가 나오는 경우
            temp_vect = feature_vector[ss]  # get 1 vector 4096-d from 88x4096

        else:
            temp_vect = feature_vector[ss:ee].mean(axis=0) # 각 열의 평균값을 가진 (1, 4096) 추출
                                                           # multiple rows vector -> one row vector
        # NORM-2 (L2)
        temp_vect = np.round(temp_vect / np.linalg.norm(temp_vect), 6)
        # divide by norm-2 (L2) of vector (Euclidean norm)=cumsum(sqrt(x[i]^2))

        if np.linalg.norm(temp_vect) == 0:
            print('Error5')

        count = count + 1 # next segment (max=32)
        Segments_Features[count]= temp_vect # push each segment feature to final 32 video segments feature



    # save 32 segment features in text file ( You can directly save and load .mat file in python as well).
    print(Segments_Features)
    print(Segments_Features.shape)

    # for i in range(0,Segments_Features.shape[0]):
    #     feat_text = str(Segments_Features[i].tolist())
    #     fid1.write(feat_text)
    #     fid1.write('\n')
    np.savetxt(txt_file_name, Segments_Features, fmt="%.6f")

    # fid1.close()

v_ApplyEyeMakeup_g01_c01
[[0.e+00 0.e+00 0.e+00 ... 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 ... 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 ... 0.e+00 0.e+00 0.e+00]
 ...
 [0.e+00 4.e-06 0.e+00 ... 0.e+00 0.e+00 0.e+00]
 [0.e+00 4.e-06 0.e+00 ... 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 ... 0.e+00 0.e+00 0.e+00]]
(32, 4096)
v_BaseballPitch_g01_c01
[[0.009141 0.       0.029037 ... 0.       0.       0.      ]
 [0.009141 0.       0.029037 ... 0.       0.       0.      ]
 [0.009141 0.       0.029037 ... 0.       0.       0.      ]
 ...
 [0.       0.001402 0.018489 ... 0.       0.       0.      ]
 [0.       0.001402 0.018489 ... 0.       0.       0.      ]
 [0.       0.001402 0.018489 ... 0.       0.       0.      ]]
(32, 4096)
